## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
bene_col = application_df.copy().drop(columns=['EIN','NAME'])
bene_col.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
bene_col.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
frec = bene_col['APPLICATION_TYPE'].value_counts()
print(frec)

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = frec[frec < 528].index
#application_types_to_replace

# Replace in dataframe
for app in application_types_to_replace:
  #print(app)
  bene_col['APPLICATION_TYPE'] = bene_col['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
bene_col['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
frec = bene_col['CLASSIFICATION'].value_counts()
print(frec)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
frec_clas = frec[frec > 1]
frec_clas

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = frec[frec < 1882].index

# Replace in dataframe
for cls in classifications_to_replace:
    bene_col['CLASSIFICATION'] = bene_col['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
back_up1 = bene_col
bene_col['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
#application_df.dtypes
cat = pd.get_dummies(bene_col[['APPLICATION_TYPE','AFFILIATION','CLASSIFICATION','USE_CASE','ORGANIZATION','INCOME_AMT','SPECIAL_CONSIDERATIONS']])
# concatenate the dataframes
bene_col = pd.concat([bene_col, cat], axis=1).reindex(bene_col.index)
#removing columns duplicated from the df
bene_col.drop(['APPLICATION_TYPE','AFFILIATION','CLASSIFICATION','USE_CASE','ORGANIZATION','INCOME_AMT','SPECIAL_CONSIDERATIONS'], axis=1, inplace=True)
back_up2 = bene_col
bene_col

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [10]:
# Split our preprocessed data into our features and target arrays
X=bene_col.drop(columns='IS_SUCCESSFUL')
y=bene_col['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=40, activation="relu", input_dim=43))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=50, activation="relu"))
#nn.add(tf.keras.layers.Dense(units=100, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 40)                1760      
                                                                 
 dense_1 (Dense)             (None, 50)                2050      
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 3,861
Trainable params: 3,861
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [14]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=30)

Epoch 1/30
804/804 [==============================] - 8s 7ms/step - loss: 0.5732 - accuracy: 0.7188
Epoch 2/30
804/804 [==============================] - 3s 3ms/step - loss: 0.5567 - accuracy: 0.7283
Epoch 3/30
804/804 [==============================] - 2s 3ms/step - loss: 0.5536 - accuracy: 0.7301
Epoch 4/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5522 - accuracy: 0.7294
Epoch 5/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5503 - accuracy: 0.7315
Epoch 6/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5496 - accuracy: 0.7325
Epoch 7/30
804/804 [==============================] - 2s 3ms/step - loss: 0.5485 - accuracy: 0.7325
Epoch 8/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5480 - accuracy: 0.7318
Epoch 9/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5473 - accuracy: 0.7329
Epoch 10/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5462 - accuracy: 0.7333

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5529 - accuracy: 0.7284 - 547ms/epoch - 2ms/step
Loss: 0.5528759956359863, Accuracy: 0.728396475315094


In [16]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity.h5')

# Optimize the Model

In [17]:
# Drop unnecesary columns
bene_col1 = back_up2.copy().drop(columns=[#'STATUS',
                                          'APPLICATION_TYPE_Other',
                                          #'APPLICATION_TYPE_T10','APPLICATION_TYPE_T19',
                                          #'APPLICATION_TYPE_T3',
                                          #'APPLICATION_TYPE_T4',
                                          #'APPLICATION_TYPE_T5',
                                          #'APPLICATION_TYPE_T6',
                                          #'APPLICATION_TYPE_T7',
                                          #'APPLICATION_TYPE_T8',
                                          #'AFFILIATION_CompanySponsored',
                                          #'AFFILIATION_Family/Parent',
                                          #'AFFILIATION_Independent',
                                          #'AFFILIATION_National',
                                          'AFFILIATION_Other',
                                          #'AFFILIATION_Regional',
                                          #'CLASSIFICATION_C1000',
                                          #'CLASSIFICATION_C1200',
                                          #'CLASSIFICATION_C2000',
                                          #'CLASSIFICATION_C2100',
                                          #'CLASSIFICATION_C3000',
                                          'CLASSIFICATION_Other'
                                          ])
#bene_col1.drop(['APPLICATION_TYPE','AFFILIATION','CLASSIFICATION','USE_CASE','ORGANIZATION','SPECIAL_CONSIDERATIONS'], axis=1, inplace=True)
#cat = pd.get_dummies(bene_col1[['INCOME_AMT']])
# concatenate the dataframes
#bene_col1 = pd.concat([bene_col1, cat], axis=1).reindex(bene_col1.index)
#bene_col1.drop(['INCOME_AMT'], axis=1, inplace=True)
bene_col1

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [18]:
# Split our preprocessed data into our features and target arrays
X=bene_col1.drop(columns='IS_SUCCESSFUL')
y=bene_col1['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [19]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
# scaler = skl.preprocessing.StandardScaler()
scaler = StandardScaler()
# The scaler is "learning" from the train dataset
# This is because the standarscaler transformation is part of my model
scaler.fit(X_train)
X_train_sc = scaler.transform(X_train)
X_test_sc = scaler.transform(X_test)
# Create the Neural Network
model = tf.keras.models.Sequential()
# Add the first hidden layer (and the input layer)
model.add(tf.keras.layers.Dense(units=100, input_dim=40, activation='sigmoid'))
# Add more hidden layers... as much as you want...
model.add(tf.keras.layers.Dense(units=50, activation='tanh'))
model.add(tf.keras.layers.Dense(units=20, activation='tanh'))
model.add(tf.keras.layers.Dense(units=20, activation='relu'))
model.add(tf.keras.layers.Dense(units=20, activation='relu'))
# Add the output layer...
# In classification problems, you need one neuron per each possible label
# In this particular case (Binary label) we just need to have one neuron with activation function of sigmoid
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid')) # Binary label prediction
#model.add(tf.keras.layers.Dense(units=20, activation='relu'))

model.compile(
    loss = 'binary_crossentropy', # This loss function is needed for Binary label prediction
    optimizer='adam', # The algorith for gradient descent
    metrics=['accuracy'] # The metrics to improve the model
)

model.fit(X_train_sc, y_train, epochs=20)
model.evaluate(X_test_sc, y_test)


Epoch 1/20
804/804 [==============================] - 9s 7ms/step - loss: 0.5833 - accuracy: 0.7155
Epoch 2/20
804/804 [==============================] - 7s 8ms/step - loss: 0.5658 - accuracy: 0.7238
Epoch 3/20
804/804 [==============================] - 5s 6ms/step - loss: 0.5610 - accuracy: 0.7262
Epoch 4/20
804/804 [==============================] - 4s 5ms/step - loss: 0.5592 - accuracy: 0.7266
Epoch 5/20
804/804 [==============================] - 5s 6ms/step - loss: 0.5571 - accuracy: 0.7299
Epoch 6/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5563 - accuracy: 0.7287
Epoch 7/20
804/804 [==============================] - 2s 3ms/step - loss: 0.5548 - accuracy: 0.7305
Epoch 8/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5543 - accuracy: 0.7288
Epoch 9/20
804/804 [==============================] - 2s 3ms/step - loss: 0.5534 - accuracy: 0.7303
Epoch 10/20
804/804 [==============================] - 3s 4ms/step - loss: 0.5526 - accuracy: 0.7317

[0.5531809329986572, 0.7293294668197632]

In [20]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimization.h5')
